In [ ]:
# Parameters (do not containerize this cell)
param_data_filename <- "Template_MBO_Example_raw.xlsx"
param_metadata_sheet <- "METADATA"
param_data_sheet <- "BIRDS"
param_user_name <- "koen.greuell@lifewatch.eu" #"See URL: beta.naavre.net/jupyter/user/[param_user_name]/lab"
param_use_dummy_data <- 1
param_years <- 7
param_latitude_north <- 90.0000
param_latitude_south <- 25.0000
param_longitude_east <- 70.0000
param_longitude_west <- 00.0000
param_upper_limit_max_depth <- 0
param_lower_limit_max_depth <- 50
param_upper_limit_min_depth <- 0
param_lower_limit_min_depth <- 50
param_first_month <- 1
param_last_month <- 3
param_output_samples_ecological_parameters <- 0
param_make_plots <- 1
param_transform_to_log10 <- 1
conf_temporary_data_directory <- "/tmp/data"
conf_virtual_lab_biotisan_euromarec <- "vl-biotisan-euromarec"
conf_minio_endpoint <- "scruffy.lab.uvalight.net:9000"
conf_minio_region <- "nl-uvalight"
conf_minio_public_bucket <- "naa-vre-public"
conf_minio_user_bucket <- "naa-vre-user-data"

In [ ]:
# Secrets (do not containerize this cell)
library("SecretsProvider")

secretsProvider <- SecretsProvider()
secret_minio_access_key = ""
secret_minio_access_key = secretsProvider$get_secret("secret_minio_access_key")
secret_minio_secret_key = ""
secret_minio_secret_key = secretsProvider$get_secret("secret_minio_secret_key")

In [ ]:
# MinIO data retriever
library("readxl")
library("aws.s3")

Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# Ensure the temporary data storage directory exists
dir.create(conf_temporary_data_directory, showWarnings = FALSE)

# Download file from bucket S3
if (param_use_dummy_data) {
        file_path <- paste(conf_virtual_lab_biotisan_euromarec, param_data_filename, sep="/")
        print(sprintf("Using dummy data for testing purposes. Set param_use_dummy_data to 0 to use your own data. Downloading data from bucket: %s folder: %s", conf_minio_public_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_public_bucket, object=file_path, file=paste(conf_temporary_data_directory, param_data_filename, sep="/"))
    } else {
        file_path <- paste(param_user_name, param_data_filename, sep="/")
        print(sprintf("Downloading data from bucket: %s folder: %s", conf_minio_user_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_user_bucket, object=file_path, file=paste(conf_temporary_data_directory, param_data_filename, sep="/"))
}

# Load data & metadata
metadata <- read_excel(paste(conf_temporary_data_directory, param_data_filename, sep="/"), sheet = param_metadata_sheet) #Load metadata sheet
data <- read_excel(paste(conf_temporary_data_directory, param_data_filename, sep="/"), sheet = param_data_sheet) #Load data sheet

# Validate column names
validate_column_names <- function(required_column_names, dataframe) {
    if (!all(required_column_names %in% colnames(dataframe))) {
    missing_columns <- required_column_names[!required_column_names %in% colnames(dataframe)]
    not_required_columns <- colnames(dataframe)[!colnames(dataframe) %in% required_column_names]
    stop(paste("\n", deparse(substitute(dataframe)), "does not contain all required column names. \n The following columns are missing: ", paste(missing_columns, collapse=", "),
              "\n The following column names are present but not required: ", paste(not_required_columns, collapse=", ")))
    } else {
    sprintf("%s has the required column names.", param_data_filename)
    }
}

required_data_columns <- c(
    'datecollected',
    'siteid',
    'sampleid',
    'basisofrecord',
    'minimumdepthinmeters',
    'maximumdepthinmeters',
    'taxaname',
    'taxanameid',
    'samplingeffort',
    'parameter',
    'parameter_value',
    'parameter_standardunit',
    'wetweightgrams',
    'dryweigthgrams',
    'afdw'
)
required_metadata_columns <- c(
    'siteid',
    'decimallatitude', 
    'decimallongitude'
)
# validate_column_names(required_data_columns, data)
validate_column_names(required_metadata_columns, metadata)

# Write (meta)data to files
metadata_as_csv_filename <- "metadata.csv"
data_as_csv_filename <- "data.csv"
metadata_from_excel_path <- paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/")
data_from_excel_path <- paste(conf_temporary_data_directory, data_as_csv_filename, sep="/")
print(sprintf("Storing metadata in: %s, and data in %s", metadata_from_excel_path, data_from_excel_path))
write.csv(metadata, file = metadata_from_excel_path)
write.csv(data, file =  data_from_excel_path)

In [ ]:
# Parameter validator
Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

upload_file_to_bucket <- function(filename) {
    aws.s3::put_object(bucket=conf_minio_user_bucket, file=filename, object=paste(param_user_name, filename, sep="/"))
}

number_of_validation_errors <- 0
current_datetime <- gsub("\\s", "_", format(Sys.time(), "%Y%m%dT%H%M%SZ"))
validation_log_file <- paste(current_datetime, "validation_log.txt", sep="_")

report_validation_error <- function(validation_failure_message) {
    number_of_validation_errors <<- number_of_validation_errors + 1
    cat(validation_failure_message, file = validation_log_file, append = TRUE)
    message(sprintf(validation_failure_message))
    return(number_of_validation_errors)
}

validate_is_number_between <- function(number, minimum, maximum) {
   if (!(is.numeric(number) & (number >= minimum) & (number <= maximum))) {
       report_validation_error(sprintf("\n %s (value: %s) is not: numeric and between %i and %i.", deparse(substitute(number)), number, minimum, maximum))
    } else {
        message(sprintf("%s = %i passed validation: It is an integer between %i and %i", deparse(substitute(number)), number, minimum, maximum))
    } 
}

validate_is_coordinate <- function(coordinate, pattern) {
    is_valid_format <- grepl(pattern, coordinate)
    if (!is_valid_format) {
      report_validation_error(paste0("The coordinate: ", coordinate, " does not follow the required ", deparse(substitute(pattern)), ": ", pattern, " e.g. -56.0953"))
    } else {
      message(sprintf("%s = %s passed validation: It is valid geographic coordinate input.", deparse(substitute(coordinate)), coordinate))
    }
}

validate_is_number_between(param_years, 0, 1000)

latitude_pattern <- "^-?((90(\\.0+)?)|([0-8]?\\d)(\\.\\d+)?)$"
longitude_pattern <- "^-?((1[0-7]\\d(\\.\\d+)?)|(180(\\.0+)?)|(\\d{1,2}(\\.\\d+)?))$"
validate_is_coordinate(param_latitude_south, latitude_pattern)
validate_is_coordinate(param_latitude_north, latitude_pattern)
validate_is_coordinate(param_longitude_east, longitude_pattern)
validate_is_coordinate(param_longitude_west, longitude_pattern)

validate_is_number_between(param_upper_limit_max_depth, -1000, 1000)
validate_is_number_between(param_lower_limit_max_depth, -1000, 1000)
validate_is_number_between(param_upper_limit_min_depth, -1000, 1000)
validate_is_number_between(param_lower_limit_min_depth, -1000, 1000)

validate_is_number_between(param_first_month, 1, 12)
validate_is_number_between(param_last_month, 1, 12)

if (number_of_validation_errors > 0) {
    upload_file_to_bucket(validation_log_file)
    stop(sprintf("Parameter validation failed. See %s for errors", validation_log_file))
}    

In [ ]:
# Species occurence data cleaner
library(dplyr)
library(tidyr)

print(paste(deparse(substitute(number_of_validation_errors)), number_of_validation_errors, sep=" = "))

validate_dataframe_has_data <- function(dataframe, dataframe_name) {
    if (nrow(dataframe) == 0) {
    stop(paste0(dataframe_name, " has no rows (0 rows). Halting execution."))
  } else {
    sprintf("%s has %i rows.", dataframe_name, nrow(dataframe))
    }
}

# Report on parameters set
print(sprintf("Filtering sites with %i or more years of data.", param_years))
print(sprintf("Filtering for data within the coordinates %s south, %s north, %s east and %s west.", param_latitude_south, param_latitude_north, param_longitude_east, param_longitude_west))
print(sprintf("Filtering for data within upper_limit_max_depth %i, lower_limit_max_depth %i, upper_limit_min_depth %i, lower_limit_min_depth %i.", param_upper_limit_max_depth, param_lower_limit_max_depth, param_upper_limit_min_depth, param_lower_limit_min_depth))
print(sprintf("Filtering for data between the month %i and month %i.", param_first_month, param_last_month))

# Assign dummy variables to prevent false Input/Output detection by the NaaVRE cell analyzer
datecollected = ""
siteid = ""
decimallatitude = ""
decimallongitude = ""

# Read (meta)data from files
md <- read.csv(paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/"), sep=",")
data <- read.csv(paste(conf_temporary_data_directory, data_as_csv_filename, sep="/"), sep=",")
validate_dataframe_has_data(data, "data read from csv")
validate_dataframe_has_data(md, "metadata read from csv")

###### Years filter ######
###### The "number of sampled years" could be changed by the user (default = 7) ######
# Create a table with sites with more than 7 sampling years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

md <- merge(md,sites, by = "siteid")
validate_dataframe_has_data(md, "metadata merged by siteid")
validate_dataframe_has_data(data, "data after first years filter")

###### Coordinates filter ######
###### The "geographical boundaries" could be changed by the user (default = latitude (25:90), longitude (-45:70)). The default values correspond to European continental waters. ######
# Keep sites within the study area [our boundaries are latitude (25:90), longitude (-45:70)]
metadata_coordinates <- md %>% select(siteid, decimallatitude, decimallongitude)
print(metadata_coordinates)

md <- dplyr::filter(md, decimallatitude >= param_latitude_south, decimallatitude <= param_latitude_north, 
                 decimallongitude >= param_longitude_west, decimallongitude <= param_longitude_east)
print(paste0("Number of sites found within the specified geolocation: ", nrow(md)))

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)
validate_dataframe_has_data(data, "data filtered by coordinates")

###### Depth filter ######
###### The "depth" could be changed by the user (default = 0 to 50 meters). It should be given in absolute value. ######
#(in this case is not necessary, but for other taxonomic groups is possible that the sample were taken at different depths. The code keeps the NAs in case that information is not known)
data <- data %>% filter((maximumdepthinmeters >= param_upper_limit_max_depth & maximumdepthinmeters <= param_lower_limit_max_depth) %>% tidyr::replace_na(TRUE))
data <- data %>% filter((minimumdepthinmeters >= param_upper_limit_min_depth & minimumdepthinmeters <= param_lower_limit_min_depth) %>% tidyr::replace_na(TRUE))
validate_dataframe_has_data(data, "data filtered by depth")

###### Season filter ######
###### The "season" could be changed by the user (default = 1:3). The default values correspond to winter.
# The period does not need to be three months, can be 1:12 for the whole year. It cannot choose months from different years (for example, November to January) ######
# In this case, most of the sampling campaigns were conducted in winter
# One was conducted in summer and should be removed since the sampling season is not consistent
data$month <- as.numeric(format(as.Date(data$datecollected), "%m")) # Create a column with the sampling month

season <- c(param_first_month:param_last_month)
data <- data %>%
  filter(month %in% season) #Remove those samples in non-consistent seasons (in this case keeps the months 1, 2 and 3, this is January, February and March)
validate_dataframe_has_data(data, "data filtered by season")

# Note that some time series can have more than one sampling campaign per year and even per season (not in this case)
# For our analysis, we are only keeping one sampling campaign per year

###### Years filter ######
###### Note that this step is repeated ######
# Update the table with sites with more than the number of sampled years
# After removing inconsistent sampling campaigns, some time series may become shorter than 8 years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)
md <- md %>% # Keep metadata from these sites
  filter(siteid %in% md$siteid)
validate_dataframe_has_data(data, "data filtered by years")

md_final <- md[,c(1:8)]
data_final <- data[,c(1:15)]

# Write data to files
cleaned_metadata_filename <- "metadata_Example.csv"
cleaned_data_filename <- "data_Example.csv"
cleaned_metadata_path <- paste(conf_temporary_data_directory, cleaned_metadata_filename, sep="/")
cleaned_data_path <- paste(conf_temporary_data_directory, cleaned_data_filename, sep="/")
print(sprintf("Storing metadata in: %s, and data in %s", cleaned_metadata_path, cleaned_data_path))
write.csv(md_final, file = cleaned_metadata_path)
write.csv(data_final, file =  cleaned_data_path)

In [ ]:
# Trend analyzer
library(vegan)
library(dplyr)
library(ggplot2)
library(nlme)
library(permute)

# Assign dummy variables to prevent false Input/Output detection by the NaaVRE cell analyzer
datecollected = ""
parameter_value = ""

# Load cleaned data & metadata
cleaned_metadata_path <- paste(conf_temporary_data_directory, cleaned_metadata_filename, sep="/")
cleaned_data_path <- paste(conf_temporary_data_directory, cleaned_data_filename, sep="/")
print(sprintf("Retrieving metadata from: %s, and data from %s", cleaned_metadata_path, cleaned_data_path))
md <- read.csv(cleaned_metadata_path, sep=",")
data <- read.csv(cleaned_data_path, sep=",")
data$year <- as.numeric(format(as.Date(data$datecollected), "%Y"))
colnames(data)

# Calculate community metrics
data.tax <- data %>%
  group_by(siteid, year, datecollected) %>%
  summarise(richness = n_distinct(taxaname[parameter_value > 0]), # Richness
            abundance = sum(parameter_value), # Abundance estimate
            parameter = unique(parameter),
            parameter_standardunit = unique(parameter_standardunit),
            diversity = diversity(parameter_value, index="shannon"), # Diversity
            )

samples_ecological_parameters <- data.tax

if (param_transform_to_log10) {
    data.tax$richness <- log10(data.tax$richness+1)
    data.tax$diversity <- log10(data.tax$diversity+1)
    data.tax$abundance <- log10(data.tax$abundance+1)
    }
    
# Depending on the analyses wanted, all the code can be run (for the three parameters) or just parts of it.
# Temporal analysis. Example with Richness and these 2 time series
results.richness <- data.frame(siteid = character(0), slope = numeric(0), p = numeric(0))

for (i in names(table(data.tax$siteid))) {
  x <- subset(data.tax, siteid == i)
  # We used GLS models taking into account the temporal autocorrelation
  gls_model <- gls(richness ~ year, data = x, correlation = corAR1(form = ~ 1 | year))
  slope <- coef(gls_model)[2]  
  p <- summary(gls_model)$tTable[2, 4]  
  se <- summary(gls_model)$tTable[, "Std.Error"][2]
  # Save results
  results.richness <- rbind(results.richness, data.frame(siteid = i, slope_richness = slope, p_richness = p, se_richness = se))
}

results.richness$trend_richness <- if_else(results.richness$slope_richness > 0, if_else(results.richness$p_richness <= 0.05, "Positive", "No change"), 
                                  if_else(results.richness$p_richness <= 0.05, "Negative", "No change"))

# Final results (just Richness)

final_results_richness <- merge(md,results.richness, by.x = "siteid")


# Temporal analysis. Example with Diversity and these 2 time series
results.diversity <- data.frame(siteid = character(0), slope = numeric(0), p = numeric(0))

for (i in names(table(data.tax$siteid))) {
  x <- subset(data.tax, siteid == i)
  # We used GLS models taking into account the temporal autocorrelation
  gls_model <- gls(diversity ~ year, data = x, correlation = corAR1(form = ~ 1 | year))
  slope <- coef(gls_model)[2]  
  p <- summary(gls_model)$tTable[2, 4]  
  se <- summary(gls_model)$tTable[, "Std.Error"][2]
  # Save results
  results.diversity <- rbind(results.diversity, data.frame(siteid = i, slope_diversity = slope, p_diversity = p, se_diversity = se))
}

results.diversity$trend_diversity <- if_else(results.diversity$slope_diversity > 0, if_else(results.diversity$p_diversity <= 0.05, "Positive", "No change"), 
                                           if_else(results.diversity$p_diversity <= 0.05, "Negative", "No change"))

# Final results (just Diversity)

final_results_diversity <- merge(md,results.diversity, by.x = "siteid")


# Temporal analysis. Example with Abundance and these 2 time series
results.abundance <- data.frame(siteid = character(0), slope = numeric(0), p = numeric(0))

for (i in names(table(data.tax$siteid))) {
  x <- subset(data.tax, siteid == i)
  # We used GLS models taking into account the temporal autocorrelation
  gls_model <- gls(abundance ~ year, data = x, correlation = corAR1(form = ~ 1 | year))
  slope <- coef(gls_model)[2]  
  p <- summary(gls_model)$tTable[2, 4]  
  se <- summary(gls_model)$tTable[, "Std.Error"][2]
  # Save results
  results.abundance <- rbind(results.abundance, data.frame(siteid = i, slope_abundance = slope, p_abundance = p, se_abundance = se))
}

results.abundance$trend_abundance <- if_else(results.abundance$slope_abundance > 0, if_else(results.abundance$p_abundance <= 0.05, "Positive", "No change"), 
                                           if_else(results.abundance$p_abundance <= 0.05, "Negative", "No change"))

# Final results (just Abundance)

final_results_abundance <- merge(md,results.abundance, by.x = "siteid")


# Final results (Richness & Diversity)

final_results_richness_diversity <- merge(final_results_richness,results.diversity, by.x = "siteid")

# Final results (Richness & Abundance)

final_results_richness_abundance <- merge(final_results_richness,results.abundance, by.x = "siteid")

# Final results (Diversity & Abundance)

final_results_diversity_abundance <- merge(final_results_diversity,results.abundance, by.x = "siteid")


# Final results (All metrics)

final_results_all <- merge(final_results_richness_diversity,results.abundance, by.x = "siteid")


######################################################################################################
# Plots
###### These plots show the frequency distribution of the time series along a "percentage of change" axis. ######
# Zero means that the ecological metric did not change, -100 means that it decreased from its initial level to zero and +100 that it doubled from its initial level
# The more sites included in the analysis, the more sense makes to plot the frequency distribution
# There is the possibility of plotting all the variables together and of changing the axis size to better observe the results, but this is heavily case dependent

# Richness
results.richness$percent_change <- (10^(results.richness$slope) - 1) * 100

plot.richness <- ggplot(data = results.richness, aes(x = percent_change)) +
  geom_density(alpha = 0.5, color = "#8a651b", linewidth = 1.5) +
  geom_vline(aes(xintercept = 0), color = "black", linetype = "dashed") +
  theme_classic() +
  labs(title = 'Frequency distribution - Richness',
       x = "Relative change (%)",
       y = "") +
  theme(legend.title = element_blank(),
        axis.text=element_text(size=10),
        axis.line.y = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.title.y = element_blank(),
        plot.title = element_text(size=16))+
  xlim(c(-100,100))

# Diversity
results.diversity$percent_change <- (10^(results.diversity$slope) - 1) * 100

plot.diversity <- ggplot(data = results.diversity, aes(x = percent_change)) +
  geom_density(alpha = 0.5, color = "#58326d", linewidth = 1.5) +
  geom_vline(aes(xintercept = 0), color = "black", linetype = "dashed") +
  theme_classic() +
  labs(title = 'Frequency distribution - Diversity',
       x = "Relative change (%)",
       y = "") +
  theme(legend.title = element_blank(),
        axis.text=element_text(size=10),
        axis.line.y = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.title.y = element_blank(),
        plot.title = element_text(size=16))+
  xlim(c(-25,25))

# Abundance
results.abundance$percent_change <- (10^(results.abundance$slope) - 1) * 100

plot.abundance <- ggplot(data = results.abundance, aes(x = percent_change)) +
  geom_density(alpha = 0.5, color = "#338888", linewidth = 1.5) +
  geom_vline(aes(xintercept = 0), color = "black", linetype = "dashed") +
  theme_classic() +
  labs(title = 'Frequency distribution - Abundance',
       x = "Relative change (%)",
       y = "") +
  theme(legend.title = element_blank(),
        axis.text=element_text(size=10),
        axis.line.y = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.title.y = element_blank(),
        plot.title = element_text(size=16))+
  xlim(c(-100,100))

######################################################################################################
# Storing temporary files
current_datetime <- gsub("\\s", "_", format(Sys.time(), "%Y%m%dT%H%M%SZ"))
data_filename_without_extension <- sub('\\.[^.]+$', '', param_data_filename)

# Store results as CSV to temporary file
output_dataframe_name <- deparse(substitute(final_results_all))
results_filename <- ""
results_filename <- paste0(paste(current_datetime, data_filename_without_extension, output_dataframe_name, sep="__"), ".csv") 
results_file_path <- paste(conf_temporary_data_directory, results_filename, sep="/")
print(sprintf("Storing final_results_all to %s", results_file_path))
write.csv(final_results_all, file = results_file_path)

# Store samples_ecological_parameters as CSV to temporary file
output_dataframe_name <- deparse(substitute(samples_ecological_parameters))
samples_ecological_parameters_filename <- ""
samples_ecological_parameters_filename <- paste0(paste(current_datetime, data_filename_without_extension, output_dataframe_name, sep="__"), ".csv") 
samples_file_path <- paste(conf_temporary_data_directory, samples_ecological_parameters_filename, sep="/")
write.csv(samples_ecological_parameters, file = samples_file_path)

# Store the images
save_plot_as_png <- function(graph) {
    graph_name <- gsub('\\.','_',deparse(substitute(graph)))
    graph_filename <- paste0(paste(current_datetime, data_filename_without_extension, graph_name, sep="__"), ".png")
    graph_file_path <- paste(conf_temporary_data_directory, graph_filename, sep="/")
    ggsave(graph, filename = graph_file_path, device = "png", height = 8, width = 12, units = "cm")
    return(graph_filename)
    }
plot_richness_filename <- ""
plot_richness_filename <- save_plot_as_png(plot.richness)
plot_diversity_filename <- ""
plot_diversity_filename <- save_plot_as_png(plot.diversity)
plot_abundance_filename <- ""
plot_abundance_filename <- save_plot_as_png(plot.abundance)

In [ ]:
# Output storer
Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# Upload files to bucket
upload_file_to_bucket <- function(filename) {
    aws.s3::put_object(bucket=conf_minio_user_bucket, file=paste(conf_temporary_data_directory, filename, sep="/"), object=paste(param_user_name, filename, sep="/"))
}

upload_file_to_bucket(results_filename)
if (param_output_samples_ecological_parameters) {
    upload_file_to_bucket(samples_ecological_parameters_filename)
}
if (param_make_plots) {
    upload_file_to_bucket(plot_richness_filename)
    upload_file_to_bucket(plot_diversity_filename)
    upload_file_to_bucket(plot_abundance_filename)
}